In [1]:
# 📘 SHL Evaluation Notebook Template
import os
import pandas as pd
import numpy as np
from recommender.embeddings import load_data, load_model
from recommender.logic import recommend_tests
from sklearn.metrics import average_precision_score
from tqdm import tqdm

# ---- Setup ----
model = load_model()
df, embeddings = load_data()

# ---- Load Ground Truth from CSV ----
test_df = pd.read_csv("dataset/Test_DATA.csv")

# Group by each query and collect expected assessments
query_groups = test_df.groupby("Query")["Assessments"].apply(list).to_dict()

# ---- Evaluation parameters ----
K = 5  # evaluate top-K


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

results = []

for query, relevant_names in tqdm(query_groups.items()):
    recs = recommend_tests(query, model, df, embeddings)
    top_names = recs.head(K)["name"].str.lower().tolist()

    # Mark as relevant if exact match or substring match
    y_true = [
        1 if any(rel.lower() in name for rel in relevant_names) else 0
        for name in top_names
    ]

    y_scores = recs.head(K)["similarity"].tolist()

    ap = average_precision_score(y_true, y_scores) if any(y_true) else 0.0
    recall = sum(y_true) / len(relevant_names)
    results.append({"query": query, "AP@K": ap, "Recall@K": recall})


100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


In [3]:

# ---- Results Summary ----
metrics_df = pd.DataFrame(results)
mean_ap = metrics_df["AP@K"].mean()
mean_recall = metrics_df["Recall@K"].mean()

print("\n--- Evaluation Results ---")
print(metrics_df)
print(f"\nMean Average Precision@{K}: {mean_ap:.4f}")
print(f"Mean Recall@{K}: {mean_recall:.4f}")



--- Evaluation Results ---
                                               query  AP@K  Recall@K
0  Content Writer required, expert in English and...   0.0       0.0
1  Development experience – Java or JavaScript, C...   0.0       0.0
2  Find me 1 hour long assesment for the below jo...   0.0       0.0
3  I am hiring for Java developers who can also c...   0.0       0.0
4  I am looking for a COO for my company in China...   0.0       0.0
5  I want to hire new graduates for a sales role ...   0.0       0.0
6  ICICI Bank Assistant Admin, Experience require...   0.0       0.0
7  KEY RESPONSIBITILES:\n \n\n Manage the sound-s...   0.0       0.0
8  station through appropriate creative and marke...   0.0       0.0
9  station through appropriate creative and marke...   0.0       0.0

Mean Average Precision@5: 0.0000
Mean Recall@5: 0.0000


SINGLE TEST CASE


In [4]:
# 📘 SHL Evaluation Notebook Template (Single Test Debug)

import pandas as pd
import numpy as np
from recommender.embeddings import load_data, load_model
from recommender.logic import recommend_tests
from sklearn.metrics import average_precision_score

# ---- Setup ----
model = load_model()
print(model)

df, embeddings = load_data()

# ---- Load Single Test Case from CSV ----
test_df = pd.read_csv("dataset/Test_DATA.csv")
test_df["Query"] = test_df["Query"].fillna(method="ffill")  # forward-fill missing queries

# Choose first test case only
single_query = test_df.iloc[0]["Query"]
relevant_names = test_df[test_df["Query"] == single_query]["Assessments"].dropna().tolist()

print(f"\nQuery: {single_query}")
print(f"\nGround Truth Assessments:")
for a in relevant_names:
    print(f"- {a}")

# ---- Run Recommendation ----
recs = recommend_tests(single_query, model, df, embeddings)
top_recs = recs.head(5)

print("\nTop 5 Recommendations:")
for _, row in top_recs.iterrows():
    print(f"- {row['name']} (sim: {row['similarity']:.4f})")

# ---- Compute AP@K and Recall@K ----
def normalize(text):
    return text.lower().replace(" | shl", "").strip()

top_names = top_recs["name"].str.lower().tolist()
y_true = [
    1 if any(normalize(rel) in normalize(name) for rel in relevant_names) else 0
    for name in top_names
]
y_scores = top_recs["similarity"].tolist()

ap = average_precision_score(y_true, y_scores) if any(y_true) else 0.0
recall = sum(y_true) / len(relevant_names) if relevant_names else 0.0

print(f"\nAP@5: {ap:.4f}")
print(f"Recall@5: {recall:.4f}")


/var/folders/s6/k6kx692506v9dlxfqv8_0frm0000gn/T/ipykernel_26217/4068917725.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_df["Query"] = test_df["Query"].fillna(method="ffill")  # forward-fill missing queries


<function load_model.<locals>.encode at 0x13dbb5620>

Query: I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.

Ground Truth Assessments:
- Automata - Fix (New) | SHL
- Core Java (Entry Level) (New) 
 | SHL
- Java 8 (New) | SHL
- Core Java (Advanced Level)  (New) | SHL
- Agile Software Development 
 | SHL
- Technology Professional 8.0  Job Focused Assessment |  SHL
- Computer Science (New) |  SHL

Top 5 Recommendations:
- Java Platform Enterprise Edition 7 (Java EE 7) (sim: 0.8445)
- Java 2 Platform Enterprise Edition 1.4 Fundamental (sim: 0.8442)
- Enterprise Java Beans (New) (sim: 0.8415)
- Java Frameworks (New) (sim: 0.8407)
- Java Web Services (New) (sim: 0.8398)

AP@5: 0.0000
Recall@5: 0.0000
